In [1]:
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior

Connecting shan@datajoint-rds.cyuksi65nrdq.us-east-1.rds.amazonaws.com:3306


## Query of a subject by the nickname

In [2]:
subj = subject.Subject & 'subject_nickname="IBL_17"'
subj

subject_uuid,subject_nickname nickname,sex sex,subject_birth_date birth date,ear_mark ear mark,subject_source name of source,lab_name name of lab,responsible_user username,protocol_number protocol number,subject_description
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,IBL_17,M,2018-04-23,None,None,zadorlab,valeria,1,None


# Query of water type that weekend since September

Step 1: Project the table action.WaterAdministration to only contain the information of interest together with the primary key. 
<br> With `.proj()`, users are able to restrict to certain attributes, rename the existing attributes, or create new attributes computed from the original ones. In this case, we would like to compute the day of week of water administration `water_admin_dow`

In [7]:
water_admin = action.WaterAdministration.proj('watertype_name', water_admin_dow='dayofweek(administration_time)')
water_admin

subject_uuid,administration_time date time,watertype_name,water_admin_dow calculated attribute
063bc242-80a7-4356-abfe-bc9d475fa8e1,2018-11-13 14:59:53,10% sucrose,3
122205e8-0a31-41ce-9632-30e7857a959d,2018-11-12 17:00:00,10% sucrose,2
122205e8-0a31-41ce-9632-30e7857a959d,2018-11-12 17:15:00,10% sucrose,2
122205e8-0a31-41ce-9632-30e7857a959d,2018-11-13 13:36:17,10% sucrose,3
1fce4668-e99f-4e50-9f75-961842781a23,2018-11-13 14:54:09,10% sucrose,3
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-11-13 15:44:57,10% sucrose,3
34d5b181-e7c8-4022-aab7-fce1e2640b57,2018-11-13 15:56:31,10% sucrose,3


Step 2: Restrict to the entries with `administration_time` in the weekend:

In [8]:
water_admin_weekend = water_admin & 'administration_time>"2018-08-31"' & 'water_admin_dow in (6,7)' & subj
water_admin_weekend

subject_uuid,administration_time date time,watertype_name,water_admin_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-21 14:17:00,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-28 15:23:40,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-05 13:51:50,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-12 14:25:18,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-19 14:39:40,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-26 16:38:51,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-11-16 15:39:34,Water,6


Step 3: If you want to put the subject_nickname aside, you could use `*` operator, that allows joining of two tables.

In [9]:
subj.proj('subject_nickname') * water_admin_weekend

subject_uuid,administration_time date time,subject_nickname nickname,watertype_name,water_admin_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-21 14:17:00,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-28 15:23:40,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-05 13:51:50,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-12 14:25:18,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-19 14:39:40,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-26 16:38:51,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-11-16 15:39:34,IBL_17,Water,6


Finally, if you want, you can also do the whole query in a single line:

In [10]:
(subject.Subject & 'subject_nickname="IBL_17"').proj('subject_nickname') * (action.WaterAdministration.proj('watertype_name', dow='dayofweek(administration_time)') & 'administration_time>"2018-08-31"' & 'dow in (6,7)')

subject_uuid,administration_time date time,subject_nickname nickname,watertype_name,dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-21 14:17:00,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-28 15:23:40,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-05 13:51:50,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-12 14:25:18,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-19 14:39:40,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-26 16:38:51,IBL_17,Water,6
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-11-16 15:39:34,IBL_17,Water,6


# Query of water type on Monday, Tuesday and Friday

Similarly as above:

In [11]:
subj.proj('subject_nickname') * (action.WaterAdministration.proj('watertype_name', water_admin_dow='dayofweek(administration_time)') & 'administration_time>"2018-08-31"' & 'water_admin_dow in (1, 2, 5)')

subject_uuid,administration_time date time,subject_nickname nickname,watertype_name,water_admin_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-24 15:24:05,IBL_17,Water,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-27 16:04:10,IBL_17,Water,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-01 14:14:11,IBL_17,Water,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-04 14:28:38,IBL_17,Water,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-11 14:22:02,IBL_17,Water,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-15 16:37:04,IBL_17,Water,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-18 15:54:03,IBL_17,Water,5


# Query of the weight, water intake and trial counts on Monday, Tuesday and Friday.

### Weight

Project the weight table to contain only the weight and the primary key, add attribute `wt_dow` 

In [12]:
weight = action.Weighing.proj('weight', wt_dow='dayofweek(weighing_time)')
weight

subject_uuid,weighing_time date time,weight weight,wt_dow calculated attribute
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-23 11:00:00,23.81,2
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-24 11:00:00,23.3,3
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-25 11:00:00,23.77,4
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-26 11:00:00,23.35,5
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-27 11:00:00,23.39,6
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-30 11:00:00,22.2,2
0026c82d-39e4-4c6b-acb3-303eb4b24f05,2018-07-31 11:00:00,22.14,3


Restrict to the weight that abtained on Monday, Tuesday and Friday:

In [13]:
weight_interest = weight & subj & 'weighing_time>"2018-08-31"' & 'wt_dow in (1, 2, 5)'
weight_interest

subject_uuid,weighing_time date time,weight weight,wt_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-17 15:55:00,22.94,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-20 14:48:00,23.48,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-24 13:53:00,23.16,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-27 20:29:19,23.11,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-01 18:56:00,23.94,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-04 19:02:23,23.69,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-11 18:46:41,23.75,5


Similarly, if you would like the `subject_nickname` showing aside, do the join trick again:

In [14]:
subj.proj('subject_nickname') * weight_interest

subject_uuid,weighing_time date time,subject_nickname nickname,weight weight,wt_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-17 15:55:00,IBL_17,22.94,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-20 14:48:00,IBL_17,23.48,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-24 13:53:00,IBL_17,23.16,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-27 20:29:19,IBL_17,23.11,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-01 18:56:00,IBL_17,23.94,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-04 19:02:23,IBL_17,23.69,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-11 18:46:41,IBL_17,23.75,5


Again, the whole query would be done in one line as well, giving the same results:

In [15]:
(subject.Subject & 'subject_nickname="IBL_17"').proj('subject_nickname') * (action.Weighing.proj('weight', wt_dow='dayofweek(weighing_time)') & 'weighing_time>"2018-08-31"' & 'wt_dow in (1, 2, 5)')

subject_uuid,weighing_time date time,subject_nickname nickname,weight weight,wt_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-17 15:55:00,IBL_17,22.94,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-20 14:48:00,IBL_17,23.48,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-24 13:53:00,IBL_17,23.16,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-27 20:29:19,IBL_17,23.11,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-01 18:56:00,IBL_17,23.94,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-04 19:02:23,IBL_17,23.69,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-11 18:46:41,IBL_17,23.75,5


### Water Intake

In [16]:
(subject.Subject & 'subject_nickname="IBL_17"').proj('subject_nickname') * (action.WaterAdministration.proj('watertype_name', water_intake="water_administered", dow='dayofweek(administration_time)') & 'administration_time>"2018-08-31"' & 'dow in (1, 2, 5)')

subject_uuid,administration_time date time,subject_nickname nickname,watertype_name,water_intake water administered,dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-24 15:24:05,IBL_17,Water,0.264,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-27 16:04:10,IBL_17,Water,0.288,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-01 14:14:11,IBL_17,Water,0.534,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-04 14:28:38,IBL_17,Water,0.882,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-11 14:22:02,IBL_17,Water,1.368,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-15 16:37:04,IBL_17,Water,2.202,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-18 15:54:03,IBL_17,Water,1.899,5


### Trial Counts

In [17]:
(subject.Subject & 'subject_nickname="IBL_17"').proj('subject_nickname') * (behavior.TrialSet.proj('trials_total_num', session_dow='dayofweek(session_start_time)') & 'session_start_time>"2018-08-31"' & 'session_dow in (1, 2, 5)')

subject_uuid,session_start_time start time,subject_nickname nickname,trials_total_num total trial numbers in this set,session_dow calculated attribute
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-17 15:56:12,IBL_17,169,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-20 14:48:48,IBL_17,129,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-24 14:53:18,IBL_17,200,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-09-27 15:29:10,IBL_17,209,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-01 13:56:02,IBL_17,323,2
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-04 14:02:26,IBL_17,475,5
24ccb0ca-c7cb-4cf0-844c-bd426af9bc56,2018-10-11 13:46:44,IBL_17,648,5
